In [19]:
import sys
sys.path.append('/Users/cosinga/code/sparta_dev/analysis/build/lib/')
from sparta_tools import moria, sparta
import numpy as np
import matplotlib.pyplot as plt

# paths
BASE = '/Users/cosinga/code/'
SIMPATH = '/Users/cosinga/sims/sparta_output/'
MORIA_FILE = 'moria_tree_cbol_l0063_n0256_no_cuts.hdf5'
SIMNAME = 'L0063_N0256_CBol'
SPARTA_FILE = 'sparta_no_cuts.hdf5'

mdata = moria.load(
    SIMPATH + '%s/%s'%(SIMNAME, MORIA_FILE), 
    ['mask_alive','id', 'parent_id_cat', 'parent_id_orbinf', 'x']
)

sdata = sparta.load(
    SIMPATH + '%s/%s'%(SIMNAME, SPARTA_FILE), load_halo_data = True,
    anl_match = ['hps', 'rsp'], anl_pad_unmatched = True,
    res_match = ['ifl', 'oct', 'tjy'], res_pad_unmatched = True
)

sparta.load: Loading file /Users/cosinga/sims/sparta_output/L0063_N0256_CBol/sparta_no_cuts.hdf5.
sparta.load: Loading 38867/38867 halos from SPARTA file (entire array)...
sparta.load: Matching result IDs in tracer sho, halo 3887/38867
sparta.load: Matching result IDs in tracer sho, halo 7773/38867
sparta.load: Matching result IDs in tracer sho, halo 11659/38867
sparta.load: Matching result IDs in tracer sho, halo 15545/38867
sparta.load: Matching result IDs in tracer sho, halo 19431/38867
sparta.load: Matching result IDs in tracer sho, halo 23317/38867
sparta.load: Matching result IDs in tracer sho, halo 27203/38867
sparta.load: Matching result IDs in tracer sho, halo 31089/38867
sparta.load: Matching result IDs in tracer sho, halo 34975/38867
sparta.load: Matching result IDs in tracer sho, halo 38861/38867


In [2]:
tjy = sdata['tcr_sho']['res_tjy']
ifl = sdata['tcr_sho']['res_ifl']
oct = sdata['tcr_sho']['res_oct']
hdata = sdata['halos']
boxsize = sdata['simulation']['box_size']
nhalos = len(hdata['sho_oct_first'])
nsnaps = sdata['simulation']['n_snaps']

In [20]:
# checking something for htXps
count = 0
def printt(out):
    print("\t" + str(out))
    return

for i in range(nhalos):
    ftcr = hdata['sho_ifl_first'][i]
    ltcr = ftcr + hdata['sho_ifl_n'][i]
    # if this halo has no tracers, skip
    if ftcr < 0:
        continue
    
    alive = hdata['id'][i] > 0
    host_moria_mask = np.isin(mdata['id'], hdata['id'][i, alive])
    if not np.any(host_moria_mask):
        print("no matches for sparta host ID in moria")
        continue

    host_moria_idx = np.unique(np.where(host_moria_mask)[1])
    try:
        host_moria_idx = host_moria_idx[0]
    except IndexError:
        print("moria index of host not found")

    for itcr in range(ftcr, ltcr):
        tracer_id = tjy['tracer_id'][itcr]
        tcr_moria_mask = np.isin(mdata['id'], tracer_id, assume_unique=True)
        tcr_moria_idx = np.where(tcr_moria_mask)
        try:
            if len(tcr_moria_idx[0]) > 2:
                printt("too many tracer-subhalo matches found in moria")
                continue
            tcr_moria_idx = tcr_moria_idx[1][0]
        except IndexError:
            printt("moria index of tracer not found")

        # check if host is in tracer's parent IDs
        sub_alive = mdata['mask_alive'][:, tcr_moria_idx]
        tcr_pids = mdata['parent_id_cat'][sub_alive, tcr_moria_idx]
        host_ids_alive = hdata['id'][i, :] > 0
        in_mask = np.isin(hdata['id'][i, host_ids_alive], tcr_pids[tcr_pids > 0])
        if not np.any(in_mask):
            print("tracer index, halo index")
            print(itcr, i)
            print("host IDs")
            print(hdata['id'][i, host_ids_alive])
            print(tcr_pids[tcr_pids > 0])
            
            print("does the tracer ever enter R200m?")
            host_rad = hdata['R200m'][i]
            tcr_rad = tjy['r'][itcr]
            host_tcr_mask = (host_rad > 0) & (tcr_rad > 0)
            any_in_r = np.any(host_rad[host_tcr_mask] > tcr_rad[host_tcr_mask])
            print(any_in_r)

            print()
            count += 1


    
    if count > 30:
        break

tracer index, halo index
38 149
host IDs
[ 4869  5719  6648  7678  8843 10086 11465 12979 14600 16436 18400 20517
 22820 25341 28045 30961 34127 37498 41091 44932 48981 53327 57981 62964]
[25339 57983 62966 68258 73818 79666 85838 92320]
does the tracer ever enter R200m?
True
tracer index, halo index
41 160
host IDs
[11674 13206 14849 16710 18709 20842 23169 25737 28470 31429 34639 38048
 41680 45554 49640 54033 58737 63771]
[ 28469  31428 107388 122751 131052 139643 148601 157896 167461 177529
 187941 198667 209816 221309 233247 245516 258240 271365 284855 298644]
does the tracer ever enter R200m?
False
tracer index, halo index
42 161
host IDs
[  866  1183  1550  1944  2409  2921  3514  4209  4992  5863  6799  7848
  9030 10297 11697 13230 14872 16737 18738 20884 23216 25789 28524 31489
 34704 38114 41749 45625 49716 54118 58828 63866]
[  31429 2362041 2384998 2407799 2430551 2453209 2475804 2498217 2520566
 2542905 2565161 2587339 2609319 2631184 2652980 2674782 2696519 2718204
 2739

In [3]:
v = 1
count = 0
for i in range(nhalos):
    ftcr = hdata['sho_ifl_first'][i]
    ltcr = ftcr + hdata['sho_ifl_n'][i]
    # if this halo has no tracers, skip
    if ftcr < 0:
        
        continue

    if not i == 552:
        continue
    
    print("finding host halo in moria...")
    
    alive = hdata['id'][i] >= 0
    if v > 2:
        print("comparing sparta halo IDs w/ and w/out alive condition")
        print(hdata['id'][i])
        print(hdata['id'][i, alive])
    
    host_moria_mask = np.isin(mdata['id'], hdata['id'][i, alive], assume_unique=True)
    host_moria_idx = np.unique(np.where(host_moria_mask)[1])[0]
    if v > 2:
        print("checking assume_unique = True, when finding host in moria")
        host_moria_mask_nonunique = np.isin(mdata['id'], hdata['id'][i, alive])
        
        host_moria_idx_nonunique = np.unique(np.where(host_moria_mask_nonunique)[1])[0]
        
        print(host_moria_idx, host_moria_idx_nonunique)
    
    print("host moria IDs")
    print(mdata['id'][:, host_moria_idx])
    print("host sparta IDs")
    print(hdata['id'][i, :])

    print("finding tracer's moria idx")
    # find the tracer's sparta idx
    tracer_ids = tjy['tracer_id'][ftcr:ltcr]
    for tid in tracer_ids:
        tcr_moria_mask = np.isin(mdata['id'], tid, assume_unique=True)
        tcr_moria_idx = np.where(tcr_moria_mask)[1][0]
        print("tracer ID: %d"%tid)
        print("moria IDs:")
        print(mdata['id'][:, tcr_moria_idx])

    print("confirming that host halo is in parent_ids of sub")
    print("parent_ids of tracer")
    print(mdata['parent_id_cat'][:, tcr_moria_idx])
    print("sparta ids of host")
    print(hdata['id'][i, :])
    
    # print(mdata['parent_id_cat'][:, host_moria_idx])

    print("the host halo is not in the parent_ids of the sub - where are the actual hosts?")
    pids = mdata['parent_id_cat'][:, tcr_moria_idx]
    parent_mask = np.isin(mdata['id'], pids[pids > 0], assume_unique=True)
    print("moria idxs of actual parents")
    print(np.where(parent_mask))
    print("moria idx of tracer host: %d"%host_moria_idx)
    print("halo IDs of actual parents")
    print(mdata['id'][parent_mask])

    print("checking if the host has its own parents:")
    pids = mdata['parent_id_cat'][:, host_moria_idx]
    parent_mask = np.isin(mdata['id'], pids[pids > 0], assume_unique=True)
    print(np.where(parent_mask))
    count += 1
    if count > 30:
        break
    


finding host halo in moria...
host moria IDs
[     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0  18273  20378  22662  25161  27857  30760  33908  37260  40837
  44663  48693  53026  57659  62627  67898  73440  79276  85416  91875
  98651 105734 113144 120941 129151 137655 146538 155758 165253      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0    

check raw halo catalog files -> will provide insight into how tracers are established, and why some of them might be so far away, confirm whether this is a bug in MORIA or in consistent-trees or halo finder.
look to add phantom check to sparta's pericenter finder
make plot of host/subhalo positions to get an idea for how they are jumping around